In [13]:
import numpy as np
import cv2
import os
import skimage.io
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.metrics import accuracy_score
from PIL import Image, ImageFont, ImageDraw
from IPython.display import display
import urllib.request
from matplotlib import pyplot as plt
from skimage.feature import local_binary_pattern,hog
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from skimage.transform import resize

# Task 1: Collect emojis, 20-30 categories

In [14]:
emoji_categories = {
    'smiling': ['😀', '😃', '😄', '😁', '😆'],
    'affectionate': ['🥰', '😍', '🤩', '😘', '😚'],
    'tongue': ['😋', '😛', '😜', '🤪', '😝'],
    'hands': ['🤗', '🤭', '🫢', '🫣', '🤫'],
    'neutral': ['🙃', '🫠', '🤐', '🤨', '😐'],
    'skeptical': ['😒', '🙄', '😬', '😮‍💨', '🤥'],
    'sleepy': ['😌', '😔', '😪', '😴'],
    'unwell': ['😷', '🤒', '🤕'],
    'disgusting': ['🤮','🤢','🥴'],
    'crying': ['😢','😭','🥲'],
    'scared': ['😱','😨','😰'],
    'joy': ['😈', '👿', '🤡', '👻', '👽'],
    'glasses': ['🥸','😎','🤓','🧐'],
    'doctor': ['🧑‍⚕️', '👨‍⚕️', '👩‍⚕️'],
    'student': ['🧑‍🎓', '👨‍🎓', '👩‍🎓'],
    'farmer': ['🧑‍🌾', '👨‍🌾', '👩‍🌾'],
    'cook':['🧑‍🍳', '👨‍🍳', '👩‍🍳'],
    'police':['👮‍♀️','👮‍♂️','👮'],
    'teacher':['👩‍🏫','🧑‍🏫','👨‍🏫'],
    'fruit': ['🍇', '🍈', '🍉', '🍊', '🍋'],
    'vegetable': ['🍆', '🥔', '🥕', '🌽', '🌶️'],
    'dessert': ['🍦', '🍧', '🍨', '🍩', '🍪', '🎂'],
    'drink': ['🍾', '🍷', '🍺', '🍻', '🥂', '🥃'],
    'award': ['🎖️', '🏆', '🏅', '🥇', '🥈', '🥉'],
    'geography': ['🌍', '🌎', '🌏', '🏔️', '⛰️', '🌋'],
    }

## convert emoji to images

In [15]:
def emoji_to_image(emoji: str, size: int = 64) -> np.ndarray:
    image = Image.new("L", (64,64), (255))
    font = ImageFont.truetype("C:/Users/zoezh/Desktop/COMP4423/A1/submission/NotoEmoji.ttf", 60, encoding='unic')
    draw = ImageDraw.Draw(image)
    draw.textbbox(xy=[0,0], text=emoji, font=font)
    draw.text((0, 0), emoji, fill=(0), font=font)
    return np.array(image)

Convert emoji into image / image list: X = []



Convert into labeled index: y = []

Convert into label list: keys = []


emoji list: emojis = []

In [16]:
X = [] # emoji_img_list
y = [] # label number index
keys = [] # label list
emojis = []
# len = 0 
# values = {}

for key, value in emoji_categories.items():
    keys.append(key)
    # values[key] = value
    for i in value:
        # print(i)
        emojis.append(i)
        img = emoji_to_image(i)
        img_array = np.array(img).flatten()
        X.append(img_array)
        y.append(keys.index(key)) # append label(number)
        # len += 1

# print(emojis)

labels = [] # label name for each emoji
for j in y:
    labels.append(keys[j])
# print(labels)

### Convert into np.array(X and y)

### split image, label_index, emoji, label into train dataset and val dataset


In [17]:
X = np.array(X)
y = np.array(y)

# split image, index, emoji, label into train dataset and val dataset
X_train, X_val, y_train, y_val, emoji_train, emoji_val, label_train, label_val= train_test_split(X, y, emojis, labels, test_size=0.2, random_state=42)

# Task 2: extract feature

In [18]:
feat_train = []
feat_val = []
feat_all = []
sift = cv2.SIFT_create()

def get_feat(img):
    # return hog(img)
    return sift_feat(img)
    # return gray_histogram(img)

def calc_distance(x, y):
    return L2_distance(x, y)
#     return L2_distance_sift(x, y)

def sift_feat(img):
    kps, des =  sift.detectAndCompute(img, None)
    responses = [kp.response for kp in kps]
    order = np.argsort(responses)[::-1]
    return np.array(des[order[:30]])

def gray_histogram(img: np.array, norm: bool = True) -> np.array:
    if img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist = np.array([len(img[img == i]) for i in range(256)])
    if norm:
        return hist / np.size(img)
    return hist

def L2_distance(x, y):
    return ((x - y) ** 2).sum() ** 0.5

def L2_distance_sift(x, y):
    dist = ((x[:, None] - y[None, :])**2).sum(axis=-1).min(axis=-1)
    dist.sort()
    return dist[:15].mean()

for sample in X_train:
    feat_train = get_feat(X_train)

for sample in X_val:
    feat_val = get_feat(X_val)




visualize feature


In [21]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# feat_train = np.array(feat_train)
# feat_val = np.array(feat_val)
# Assuming feat_train and feat_val are the feature vectors extracted from the images

# Combine training and validation features for visualization
feat_combined = feat_train + feat_val

# Initialize t-SNE
tsne = TSNE(n_components=2, random_state=42)

# Fit and transform the features to 2D space
feat_tsne = tsne.fit_transform(feat_combined)

# Separate back into training and validation sets
feat_train_tsne = feat_tsne[:len(feat_train)]
feat_val_tsne = feat_tsne[len(feat_train):]

# Visualize the feature space
plt.figure(figsize=(8, 6))
plt.scatter(feat_train_tsne[:, 0], feat_train_tsne[:, 1], label='Training Set', color='b')
plt.scatter(feat_val_tsne[:, 0], feat_val_tsne[:, 1], label='Validation Set', color='r')
plt.title('t-SNE Visualization of Feature Space')
plt.legend()
plt.show()

ValueError: operands could not be broadcast together with shapes (330480,) (0,) 

# Task 3: Train a classifier - SVM

In [ ]:
# Train with SVM
model = SVC(kernel= 'linear')

model.fit(X_train, y_train)

# test with SVM (self-values)
y_pred = model.predict(X_val) 
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

print('\n--- Test result with original dataset ---')
print('-----------------------------------------')
category_list = list(emoji_categories)
for val, pred, l in zip(emoji_val, y_pred, label_val):
    print(val, 'with label', l,'is predicted as', category_list[pred])

Accuracy: 54.55%

--- Test result with original dataset ---
-----------------------------------------
🍊 with label fruit is predicted as dessert
😋 with label tongue is predicted as disgusting
😆 with label smiling is predicted as smiling
🥈 with label award is predicted as award
👩‍🏫 with label teacher is predicted as farmer
👩‍🍳 with label cook is predicted as farmer
😌 with label sleepy is predicted as sleepy
😰 with label scared is predicted as scared
🥃 with label drink is predicted as drink
😛 with label tongue is predicted as tongue
🍆 with label vegetable is predicted as joy
👿 with label joy is predicted as joy
😀 with label smiling is predicted as smiling
🥔 with label vegetable is predicted as vegetable
🫣 with label hands is predicted as affectionate
⛰️ with label geography is predicted as geography
🧑‍⚕️ with label doctor is predicted as farmer
🍋 with label fruit is predicted as neutral
👨‍🍳 with label cook is predicted as farmer
🥲 with label crying is predicted as crying
😜 with label ton

# Task 4: Test the classifier using emojis that are unseen in the training and validation processes

#### first read images unseen(generated by Emoji kitchen)

In [ ]:
# Specify path to folder containing the unseen images
folder_path = 'C:/Users/zoezh/Desktop/COMP4423/A1/submission/unseen'

unseen_img_list = []
unseen_img_fname = []
# Iterate over the files in the folder
for file in os.listdir(folder_path):
    # Check if the file is an image
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Construct the full file path
        file_path = os.path.join(folder_path, file)

        # Read the image using OpenCV
        image = cv2.imread(file_path)

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Check if the image was successfully read
        if image is not None:
            # Convert the image to 64 * 64 image array and then convert color into gray, then flatten
            image = cv2.cvtColor(cv2.resize(image, (64, 64)), cv2.COLOR_BGR2GRAY).flatten()

            # Display the shape of the image array
            print("Image:", file)
            # unseen_img_fname.append(file)
            print("Image shape:", image.shape)
            # display(image)
            # plt.imshow(image_rgb)
            # plt.axis('off')  # Turn off the axis labels
            # plt.show()
        else:
            print("Failed to read the image:", file)
    
    unseen_img_list.append(image)

unseen_img_list = np.array(unseen_img_list)
        

Image: 1.png
Image shape: (4096,)
Image: 2.png
Image shape: (4096,)
Image: 3.png
Image shape: (4096,)
Image: 4.png
Image shape: (4096,)
Image: 5.png
Image shape: (4096,)
Image: 6.png
Image shape: (4096,)


#### test with model

In [ ]:
svm = SVC(kernel='linear')
svm.fit(X, y)
unseen_pred = model.predict(unseen_img_list) 

for pred in unseen_pred:
    print(category_list[pred])


drink
drink
drink
drink
drink
joy


# Task 5: Test the model using human face emotions datasets

In [ ]:
# Specify path to folder containing the unseen images
folder_path = 'C:/Users/zoezh/Desktop/COMP4423/A1/submission/face'

face_img_list = []
face_img_fname = []
# Iterate over the files in the folder
for file in os.listdir(folder_path):
    # Check if the file is an image
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Construct the full file path
        file_path = os.path.join(folder_path, file)

        # Read the image using OpenCV
        face_image = cv2.imread(file_path)

        # Check if the image was successfully read
        if image is not None:
            # Convert the image to a NumPy array
            image_array = np.array(image).flatten()

            # Display the shape of the image array
            print("Image:", file)
            # unseen_img_fname.append(file)
            print("Image shape:", image.shape)
            # display(image)
            # plt.imshow(image_rgb)
            # plt.axis('off')  # Turn off the axis labels
            # plt.show()
        else:
            print("Failed to read the image:", file)
    
    face_img_list.append(image)

face_img_list = np.array(face_img_list)

Image: 1.jpg
Image shape: (4096,)
Image: 2.jpg
Image shape: (4096,)
Image: 3.jpg
Image shape: (4096,)
Image: 4.jpg
Image shape: (4096,)
Image: 5.jpg
Image shape: (4096,)


In [ ]:
svm = SVC(kernel='linear')
svm.fit(X, y)
face_pred = model.predict(face_img_list) 

for pred in face_pred:
    print(category_list[pred])

joy
joy
joy
joy
joy
